In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
dir_data = "/content/gdrive/My Drive/Colab Notebooks/SML/dataset1"
dir_seg = dir_data + "/annotations_prepped_train/"
dir_img = dir_data + "/images_prepped_train/"

import glob, os
all_img_paths = glob.glob(os.path.join(dir_img, "*.png"))
all_mask_paths = glob.glob(os.path.join(dir_seg, "*.png"))

In [6]:
import cv2
import imageio

x = []
y = []
for i in range(len(all_img_paths)):
    img = cv2.imread(all_img_paths[i])
    img = cv2.resize(img, (224,224))
    mask_path = dir_seg + all_img_paths[i].split('/')[-1]
    img_mask = imageio.imread(mask_path)
    img_mask = cv2.resize(img_mask, (224,224))
    x.append(img)
    y.append(img_mask)

In [ ]:
import numpy as np
set(np.array(y).flatten())
n_classes = len(set(np.array(y).flatten()))
print(n_classes)

12


In [ ]:
def getsegnum(img):
  seg_labels = np.zeros((224,224,12))
  for c in range(12):
    seg_labels[:,:,c] = (img==c).astype(int)
  return seg_labels

y2 = []
for i in range(len(y)):
  y2.append(getsegnum(y[i]))

In [ ]:
x = np.array(x)/255
y2 = np.array(y2)
print(x.shape, y2.shape)
print(np.unique(y2))

(367, 224, 224, 3) (367, 224, 224, 12)
[0. 1.]


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_resnet50_unet(inputs):

    """ Pre-trained ResNet50 Model """
    resnet50 = ResNet50(include_top=False, weights="imagenet", input_shape=inputs)
    resnet50.trainable = False

    """ Encoder """
    s1 = resnet50.get_layer("input_1").output           ## (512 x 512)
    s2 = resnet50.get_layer("conv1_relu").output        ## (256 x 256)
    s3 = resnet50.get_layer("conv2_block3_out").output  ## (128 x 128)
    s4 = resnet50.get_layer("conv3_block4_out").output  ## (64 x 64)

    """ Bridge """
    b1 = resnet50.get_layer("conv4_block6_out").output  ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 224)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 112)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 56)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 28)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="ResNet50_U-Net")
    return model

if __name__ == "__main__":
    inputs = (224, 224, 3)
    model = build_resnet50_unet(inputs)
    model.summary()

ValueError: ignored

In [ ]:
model = build_resnet50_unet(inputs = (224, 224, 3))

ValueError: ignored

In [ ]:
model.compile(optimizer=Adam(1e-3, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy']) #class : 12개의 channel
history = model.fit(x,y2, epochs=10, batch_size=1, validation_split=0.1)